# Setup Kaggle API on Windows
Instructions to set up Kaggle API on a Windows device, including downloading kaggle.json and setting environment variables.

In [ ]:
# Install Kaggle package
%pip install kaggle

# Import necessary libraries
import os
import shutil

# Define the path to the kaggle.json file
kaggle_json_path = 'kaggle.json'

# Check if kaggle.json file exists
if os.path.exists(kaggle_json_path):
    # Create .kaggle directory in the user's home directory
    kaggle_dir = os.path.join(os.path.expanduser('~'), '.kaggle')
    os.makedirs(kaggle_dir, exist_ok=True)
    
    # Copy kaggle.json to the .kaggle directory
    shutil.copy(kaggle_json_path, kaggle_dir)
    
    # Set permissions for the kaggle.json file
    os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)
else:
    print("kaggle.json file not found. Please place it in the same directory as this notebook.")

# Download Dataset from Kaggle
Use Kaggle API to download the Twitter sentiment analysis dataset.

In [5]:
# Download Dataset from Kaggle

# Import necessary libraries
import kaggle

# Define the Kaggle dataset to download
dataset = 'kazanova/sentiment140'

# Download the dataset using Kaggle API
kaggle.api.dataset_download_files(dataset, path='.', unzip=True)

# Verify the dataset is downloaded
import os
if os.path.exists('training.1600000.processed.noemoticon.csv'):
    print("Dataset downloaded successfully.")
else:
    print("Failed to download dataset.")

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
Dataset downloaded successfully.


importing the dependencies

In [6]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

In [7]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [11]:
# loading dataset to pandas dataframe
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")

In [12]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


No column names

In [14]:
# naming the columns and reading the dataset again
column_names = ["target","id","date","flag","user","text"]
df = pd.read_csv("training.1600000.processed.noemoticon.csv",names=column_names, encoding="ISO-8859-1")

In [12]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
df.shape

(1600000, 6)

- 0:negative tweet
- 4:positive tweet

In [16]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

No missing values

In [17]:
df.duplicated().sum()

0

No duplicates

In [18]:
# checking the distribution of target column (how +ve and -ve tweets)
df["target"].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Hence we can use supervised learning algorithm

Convert the target "4" to "1"

In [19]:
df.replace({"target":{4:1}}, inplace=True)

In [20]:
df["target"].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

**STEMMING**

In [21]:
ps = PorterStemmer()

In [23]:
def stemming(content):
  stemmed_content = re.sub("[^a-zA-Z]"," ", content)
  # we are taking content and removing all things that are not alphabets
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
  stemmed_content = " ".join(stemmed_content)
  return stemmed_content

In [24]:
df["stemmed_content"] = df["text"].apply(stemming)

In [25]:
# saving the stemmed dataframe as a csv file (stemming took way too long so better to save the stemmed dataframe)
df.to_csv('stemmed.csv', index=False)

In [38]:
twitter = pd.read_csv("stemmed.csv", encoding="ISO-8859-1")
twitter.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [39]:
print(twitter["stemmed_content"])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [40]:
print(twitter["target"])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [51]:
twitter.isnull().sum()

target               0
id                   0
date                 0
flag                 0
user                 0
text                 0
stemmed_content    495
dtype: int64

In [52]:
twitter.dropna(inplace=True)

In [53]:
twitter.isnull().sum()

target             0
id                 0
date               0
flag               0
user               0
text               0
stemmed_content    0
dtype: int64

In [56]:
twitter.shape

(1599505, 7)

In [57]:
# separating the data and label
X = twitter["stemmed_content"].values
y = twitter["target"].values

In [58]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [59]:
print(y)

[0 0 0 ... 1 1 1]


In [60]:
X.shape

(1599505,)

In [61]:
y.shape

(1599505,)

In [62]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=2)
# stratify = y will make sure that split has equal distribution of 0 and 1 in training data and test data

In [63]:
print(X_train.shape, X_test.shape)

(1279604,) (319901,)


In [64]:
# converting the text data to numerical data based on how many times each word had been repeated
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [65]:
print(X_train)

  (0, 338364)	0.7628381862700486
  (0, 152969)	0.3776275607531222
  (0, 365681)	0.41460113464293163
  (0, 410810)	0.3218403736026031
  (1, 384003)	0.3691429868522367
  (1, 441346)	0.6830798699695361
  (1, 28692)	0.5483731943180142
  (1, 150491)	0.3105192204255133
  (2, 332366)	0.7657519561732702
  (2, 157423)	0.5219552654647517
  (2, 95030)	0.37574811040167677
  (3, 80636)	0.3835292297173858
  (3, 135944)	0.4063031361036051
  (3, 445011)	0.2711595990651876
  (3, 120080)	0.293733848490576
  (3, 226721)	0.3566587252607362
  (3, 285969)	0.188883972260186
  (3, 67987)	0.24270801629409655
  (3, 295762)	0.29911337364690993
  (3, 301319)	0.17828517423734444
  (3, 13721)	0.2401642375246812
  (3, 170856)	0.16812262000788783
  (3, 143888)	0.3146866968657337
  (4, 178224)	0.43523912206826365
  (4, 320766)	0.24515767832411012
  :	:
  (1279601, 217405)	0.2861389702815746
  (1279601, 359279)	0.34642256117664655
  (1279601, 454001)	0.2515835426017201
  (1279601, 59311)	0.28600231966534617
  (1279601,

In [34]:
print(X_test)

  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538
  (319995, 107868)	0.33399349737546963
  (319995, 109379)	0.3020896484890833
  (319995, 155493)	0.2770682832971669
  (319995, 2133

Training the model

In [66]:
model = LogisticRegression(max_iter=1000)

In [67]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [68]:
train_accuracy = accuracy_score(y_train, model.predict(X_train))
print(f"Training accuracy = {train_accuracy*100. :2f}%")

Training accuracy = 80.242169%


In [69]:
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print(f"Test accuracy = {test_accuracy*100. :2f}%")

Test accuracy = 77.832204%


Saving the trained model

In [70]:
import pickle
filename = "trained_model.sav"
pickle.dump(model, open(filename, "wb"))

Using the saved model for future predictions

In [72]:
# loading the model
load_model = pickle.load(open("trained_model.sav","rb"))

In [ ]:
X_new = X_test[560]
print("actual prediction",y_test[560])
prediction = model.predict(X_new)
# print(prediction)

if prediction[0] == 0:
  print("Negative tweet")
else:
  print("Positive tweet")

actual prediction 0
Negative tweet
